# Imports

In [ ]:
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta as td
from pathlib import Path
from plotly import graph_objects as go
from hbt_tools import plots as hp
from hbt_tools.utils_hbt import mdb_getdata

# Settings

In [ ]:
inputs_dir = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\13 Nachrüstung RÜ\Datenauswertung\Monatsdaten")

save_dir = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\13 Nachrüstung RÜ\Datenauswertung\Plots")

idx_niveaus = [0,3,6,9]

prop_dict = {
    'RÜ Schulstrasse; Niveau Istwert; A04.MA.11; Einheit: cm': {
        'h_entlastung': 121
    },

    'RÜ Löchliweg; Niveau Istwert; A16.MA.01; Einheit: cm': {
        'h_entlastung': 37
    },

    'RÜ Bodenstrasse; Niveau Istwert; A15.MA.01; Einheit: cm': {
        'h_entlastung': 30
    },

    'RÜ Mülenen; Niveau Istwert; A11.MA.01; Einheit: cm': {
        'h_entlastung': 0
    }
}


# Read data

## Rain data

In [ ]:
id = 1780
start_date = dt(2023, 3, 1)
end_date = dt(2023, 12, 1)
rain = mdb_getdata(id, start_date, end_date)

## PLS Exports

In [ ]:
files_list = list(inputs_dir.glob('*.csv'))

In [ ]:
datumstr_list = sorted([x.stem.split('_')[-1] for x in files_list])
datum1 = datumstr_list[0]
datum2 = datumstr_list[-1]

In [ ]:
date_format = '%d.%m.%Y %H:%M'
data = []
for file in files_list:
    df = pd.read_csv(file, sep=';', header='infer', index_col=0, parse_dates=[0],
        date_format=date_format)
    cols_dict = {}
    for col_name_old in df.columns:
        col_name_new = col_name_old.replace('\t', '; ')
        cols_dict.update({col_name_old: col_name_new})
    df.rename(columns=cols_dict, inplace=True)

    data.append(df)
data = pd.concat(data)
data.sort_index(inplace=True)

## Wiesengrundstrasse

In [ ]:
# Wiesengrundstrasse
file_path = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\13 Nachrüstung RÜ\Datenauswertung\Exports_XLS\Wiesengrundstrasse Surverse.xls")

wiesengrund_data = pd.read_excel(file_path, sheet_name='data')
wiesengrund_data.set_index('Date', inplace=True)


wiesengrund_surverse = pd.read_excel(file_path, sheet_name='surverse')
wiesengrund_surverse.set_index('Date', inplace=True)

def str_to_td(td_str):
    delta_t = dt.strptime(td_str, '%H:%M:%S')
    delta_t = td(hours=delta_t.hour, minutes=delta_t.minute, seconds=delta_t.second)
    delta_t = delta_t.total_seconds() / 60 # sec -> min
    return delta_t

wiesengrund_surverse.iloc[:,2] = wiesengrund_surverse.iloc[:,2].apply(lambda x: str_to_td(x))

# Plots

## Niveaus zusammen

In [ ]:
niveaus = data.iloc[:,idx_niveaus]
fig = go.Figure()
for name in niveaus:
    fig.add_trace(go.Scatter(x=niveaus.index, y=niveaus[name], name=name))

fig.update_xaxes(title='Zeit')
fig.update_yaxes(title='Niveau [cm]')

save_path = save_dir / f'Niveaus_alle_{datum1}-{datum2}.html'
fig.write_html(save_path)

## Niveaus separat

In [ ]:
# 1 Plot / Messung
for col in prop_dict.keys():
    fig = go.Figure()
    df = data[col]

    h_entl = prop_dict[col]['h_entlastung']

    fig.add_trace(go.Scatter(x=df.index, y=df.iloc[:], name=col))
    fig.add_hline(y=h_entl, line_width=0.5)
    fig.add_annotation(x=df.index[-1], y=h_entl, text=f'Entlastung = {h_entl} cm')

    fig.update_xaxes(title='Zeit')
    fig.update_yaxes(title='Niveau [cm]')

    save_path = save_dir / (col.split(';')[0] + '.html')
    fig.write_html(save_path)

## Anzahl Entlastungen

In [ ]:
n_entlastungen = data.iloc[:,[1,4,7,10,12]]
fig = go.Figure()
for name in n_entlastungen:
    fig.add_trace(go.Scatter(x=n_entlastungen.index, y=n_entlastungen[name], name=name))

fig.update_xaxes(title='Zeit')
fig.update_yaxes(title='Anzahl Entlastungen')

save_path = save_dir / f'Anzahl-Entlastungen_alle_{datum1}-{datum2}.html'
fig.write_html(save_path)

## Wiesengrundstrasse

In [ ]:
# Aktivierung
wg_active = wiesengrund_data.loc[:, 'Surverse[0]']

save_path = save_dir / 'Wiesengrund_On-Off.html'
hp.ply_2y(dfs1=wg_active, dfs2=rain, names=['Wiesengrund on/off', 'Regen WAE'], xlabel='Zeit',
    ylabels=['On / Off', 'Niederschlag [mm]'], save_path=save_path);

In [ ]:
# Überlaufzeit
save_path = save_dir / 'Wiesengrund_Überlaufzeit.html'
names = ['Überlaufzeit', 'Überlaufzeit kumulativ', 'Regen WAE']
hp.ply_2y([wiesengrund_surverse.iloc[:,2], wiesengrund_surverse.iloc[:,2].cumsum()], rain, xlabel='Zeit',
    ylabels=['Überlaufzeit [min]', 'Niederschlag [mm]'], names=names, save_path=save_path);